## <center>循环神经网络应用</center>

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
gpu_no = '0'
os.environ["CUDA_VISIBLE_DEVICES"] = gpu_no
# 定义TensorFlow配置
config = tf.ConfigProto()
# 配置GPU内存分配方式，按需增长，很关键
config.gpu_options.allow_growth = True
# 配置可使用的显存比例
config.gpu_options.per_process_gpu_memory_fraction = 0.3
# 在创建session的时候把config作为参数传进去
sess = tf.Session(config = config)

### 1 初始化 RNN

在 RNN 的计算过程中,需要一个初始的隐含状态,一般就使用如下两种初始化方法:
#### (1) 初始化为0
对于正向或反向,第一个 cell 传入时,没有之前序列的输出值,所以需要对其进行初始化.一般来讲,如果不指定初始状态,那么一定要指定 dtype ,可以不用刻意指定初值状态,系统默认初始化为 0.
```python
# 在后续的 cell 实例化中,将 initial_state 传入即可
initial_state = lstm_cell.zero_state(batch_size,dtype)
```
#### (2) 初始化为指定值
在确保创建组成 RNN 的 cell 时, 设置了输出为元组类型(创建 cell 类的初始化参数 state_is_tuple=True) 的前提下,可以使用 LSTMStateTuple 函数.
```python
from tensorflow.contrib.rnn.python.ops.core_rnn_cell_impl import LSTMStateTuple
c_state = ...
h_state = ...
# c_state , h_state 都为 Tensor
initial_state = LSTMStateTuple(c_state,h_state)
```

### 2 优化 RNN
RNN 的优化技巧有很多,对于前面讲述的神经网络技巧大部分在 RNN 中都适用,但也有例外.下面介绍两个 RNN 特有的优化方法.

#### (1) dropout 功能
在 RNN 中,如果想使用 dropout 功能,不能用以前的 CNN 下的 dropout, 在 CNN 中:
```python
def dropout(x,keep_prob,noise_shape = None,seed=None,name=None)
```
因为 RNN 有自己的 dropout,并且实现方式与 RNN 不一样:
```python
def rnn_cell.DropoutWrapper(rnn_cell,input_keep_prob=1.0,output_keep_prob=1.0)
```
使用举例:
```python
lstm_cell = tf.nn.rnn_cell.DropoutWrapper(lstm_cell,output_keep_prob=0.5)
```
从 $t-1$ 时刻的状态传递到 $t$ 时刻进行计算,这中间不进行 memory 的 dropout,仅在同一时刻 $t$ 中,多层 cell 之间传递信息进行 dropout. 所以,RNN 的 dropout 方法会有两个设置参数 input_keep_prob (传入 cell 的保留率) 和 output_keep_prob (输出 cell 的保留率)

+ 如果希望是 input 传入 cell 时丢弃掉一部分 input 信息,就设置 input_keep_prob,那么传入到 cell 的就是部分 input
+ 如果希望 cell 的 output 只有一部分作为下一层 cell 的输入,就定义 output_keep_prob

示例代码如下:
```python
lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(n_hidden,forget_bias=1.0,state_is_tuple=True)
lstm_cell = tf.nn.rnn_cell.DropoutWrapper(lstm_cell,output_keep_prob=0.5)
```
在上面代码中,一个 RNN 层后面跟一个 DropoutWrapper,是一种常见的用法.

#### (2) LN 基于层的归一化
在 RNN 这种特殊的结构中,不能直接使用 BN:
+ 在 BN 中, 每一层的输入只考虑当前批次样本 (或批次样本的转化值) 即可.
+ 在 RNN 中,每一层的输入处理当前批次样本转化值,还要考虑样本中上一个序列样本的输出值,所以对于 RNN 的归一化,BN 算法不再适用,最小批次覆盖不了全部的输入数据,而是需要对于输入 BN 的某一层来做归一化, 也就是 layer normalization.

### 3 CTC 网络的 loss
CTC 网络的 loss 就不能用平方差,更不能使用交叉熵,它有一个更复杂的计算公式.

#### (1) ctc_loss 函数
在 Tensorflow 中提供了一个 ctc_loss 函数,其作用就是按照序列来处理输出标签和标准标签之间的损失.
```python
tf.nn.ctc_loss(labels,inputs,sequence_length,preprocess_collapse_repeated,ctc_merge_repeated)
labels: 一个  int32 类型的稀疏矩阵张量 (SparseTensor)
inputs: 常用变量 logits 表示, 经过 RNN 后输出的标签预测值,三维浮点张量,当 time_major 为 false 时,形状为 [batch_size,timesteps,num_classes], 否则为 [timesteps,batch_size,num_classes]
sequence_length: 序列长度
preprocess_collapse_repeated: 是否需要预处理,将重复的 label 删除成一个 label,默认是 false
ctc_merge_repeated: 将重复标签合并,默认是 true
```
对于 preprocess_collapse_repeated 与 ctc_merge_repeated,都是对于 ctc_loss 中重复标签处理的控制.

参数情况 | 说明
 :-: | :-: 
preprocess_collapse_repeated = true <br/> ctc_merge_repeated= true| 忽略全部重复标签,只计算不重复的标签
preprocess_collapse_repeated = false <br/> ctc_merge_repeated= true| 标准的CTC模式,也是默认模式,不做预处理,只在运算时重复标签将不再当成独立的标签来计算
preprocess_collapse_repeated = true <br/> ctc_merge_repeated= false| 忽略全部重复标签,只计算不重复的标签,因为预处理时已经把重复的标签去掉了
preprocess_collapse_repeated = true <br/> ctc_merge_repeated= true| 所有重复标签都参与计算

对于 ctc_loss 的返回值,仍然属于 loss 的计算模式,当取批次样进行训练时,同样也需要对最终的 ctc_loss 求均值
<font color='red'>注意</font><br/>
对于重复标签方面的 ctc_loss 计算,一般情况下默认即可.另外这里有个隐含规则, inputs 中的 classes 是指需要输出多少类,在使用 ctc_loss 时,要将 classes+1 ,即再多生成一个类,用于存放 blank. 因为输入的序列 与 label 并不是一一对应的,所以需要通过添加 blank 类,当对应不上时,最后的 softmax 就会将其生成到 blank. 具体做法就是在最后的输出层多构建一个节点即可. 这个规则是 ctc_loss 内置的,否则当标签 label 中的类索引等于 inputs 中的 size-1 时会报错


#### (2) SparseTensor 类型
稠密矩阵就是我们常见的矩阵,当稠密矩阵中大部分的数据都是 0 时,就可以使用一种更好的存储方式,只将矩阵中不为 0 的索引和值记录下来.这种方式就可以大大节省内存空间,这就是所谓的稀疏矩阵.
稀疏矩阵在 Tensorflow 中的结构类型如下:
```python
SparseTensor(indices,values,dense_shape)

+ indices: 就是前面所说的不为 0 的位置信息,它是一个二维的 int64 tensor, 指定了 Sparse tensor 中的索引,例如, indices = [[1,3],[2,4]], 表示 dense tensor 中对应的索引 [1,3] 和 [2,4] 位置的元素值不为 0

+ values: 一个 list, 存储稠密矩阵中不为 0 位置所对应的值,它要与 indices 里的顺序对应,例如, indices = [[1,3],[2,3]], values = [18,24], 表明 [1,3] 位置的值是 18, [2,4] 位置的值是 24

+ dense_shape: 一个一维的 int64 tensor,代表原来稠密矩阵的形状
```

#### (3) 生成 SparseTensor

In [10]:
def sparse_from_dense(dense,dtype = np.int32):
    indices = []
    values = []
    for n, seq in enumerate(dense):
        indices.extend(zip([n]* len(seq),range(len(seq))))
        values.extend(seq)
    indices = np.asarray(indices,dtype = np.int64)
    values = np.asarray(values,dtype = dtype)
    shape = np.asarray([len(dense),indices.max(0)[1]+1],dtype=np.int64)
    return tf.SparseTensor(indices=indices,values=values,dense_shape=shape)
dense_matrix = np.eye(M=3,N=3,dtype=np.int32)
print(dense_matrix)
result = sparse_from_dense(dense_matrix)
print(sess.run(result))

[[1 0 0]
 [0 1 0]
 [0 0 1]]
SparseTensorValue(indices=array([[0, 0],
       [0, 1],
       [0, 2],
       [1, 0],
       [1, 1],
       [1, 2],
       [2, 0],
       [2, 1],
       [2, 2]]), values=array([1, 0, 0, 0, 1, 0, 0, 0, 1], dtype=int32), dense_shape=array([3, 3]))


#### (4) SparseTensor转dense
在 Tensorflow 中,可以方便地实现 SparseTensor 转 dense:
```python
tf.sparse_tensor_to_dense(sp_input,default_value=0,validate_indices=true)
sp_input: 一个 SparseTensor
default_value: 没有指定索引的对应的默认值,默认值为 0
validate_indices: 布尔值,如果为 true,该函数会检查 sp_input 的 indices 的索引是否存在重复
```

#### (5) levenshtein 距离
ctc_loss 是用来训练时间序列分类模型的,评估模型时,一般常使用计算得到的 levenshtein 距离作为模型的评分.

levenshtein 距离又叫编辑距离,是指两个字符串之间,由一个转换成另一个所需的最少编辑操作次数.一般来说,编辑距离越小,两个字符串的相似度越大.

在 Tensorflow 中, levenshtein 距离的处理被封装成对两个稀疏矩阵进行的操作,具体定义如下:
```python
tf.edit_distance(hypothesis,truth,normalize=true)
hypothesis: SparseTensor 类型,预测的序列结果
truth: SparseTensor 类型,真实结果
normalize: 默认为 true,求出来的编辑距离除以真实序列的长度
返回值: R-1 维的 DenseTensor,包含着每个序列的 levenshtein 距离
```

In [11]:
# 'hypothesis' is a tensor of shape `[2, 1]` with variable-length values:
#   (0,0) = ["a"]
#   (1,0) = ["b"]
hypothesis = tf.SparseTensor(
    [[0, 0, 0],
     [1, 0, 0]],
    ["a", "b"],
    (2, 1, 1))

# 'truth' is a tensor of shape `[2, 2]` with variable-length values:
#   (0,0) = []
#   (0,1) = ["a"]
#   (1,0) = ["b", "c"]
#   (1,1) = ["a"]
truth = tf.SparseTensor(
    [[0, 1, 0],
     [1, 0, 0],
     [1, 0, 1],
     [1, 1, 0]],
    ["a", "b", "c", "a"],
    (2, 2, 2))

normalize = True
result = tf.edit_distance(hypothesis=hypothesis,truth=truth,normalize=True)
print(sess.run(result))

# 'output' is a tensor of shape `[2, 2]` with edit distances normalized
# by 'truth' lengths.
# output ==> [[inf, 1.0],  # (0,0): no truth, (0,1): no hypothesis
#            [0.5, 1.0]]  # (1,0): addition, (1,1): no hypothesis

[[inf 1. ]
 [0.5 1. ]]


### 4 CTCdecoder

虽然在输入 ctc_loss 中的 logits(inputs) 是我们的预测结果,但却带有空标签,而且是一个与时间序列强对应的输出. 在实际情况下,我们需要一个转化好的类似于原始标准标签的输出.这时可以使用 CTCdecoder,经过它对预测结果的加工后,就可以与标准标签进行损失值的运算了.在 Tensorflow 中, CTCdecoder 有两个函数
```python
tf.nn.ctc_greedy_decoder(inputs,sequence_length,merge_repeated=True)

+ 使用 greedy 策略的 CTC 解码:
+ inputs: 模型的输出预测值 logits, shape 为 (timesteps * batch_size * num_classes)
+ sequence_length: 序列的长度. 该 sequence_length 和用在 dynamic_rnn 中的 sequence_length 是一致的.
+ 返回值: 是一个tuple(decoded,log_probabilities)
+ decoded: 是一个 list .只有一个元素,是一个 SparseTensor, 保存着解码的结果
+ log_probabilities: 一个浮点型矩阵 (batch_size * 1) 包含着序列的 log 概率

tf.nn.ctc_beam_search_decoder(inputs,sequence_length,beam_width=100,top_paths =1)

+ 另一种寻路策略,参数同上
```

在实际情况中,解码完事的 decoder 是 list, 不能直接用,通常取 decoder[0],然后转成密集矩阵,得到的是一个批次的结果,然后再一条一条地取得每一个样本的结果